<h1>gene

In [2]:
import numpy as np
import random, pickle
import utill, student_class, apply_subject
from basic_var import SPC, NOSD, NOB, NOSJ

class Gene:
    def __init__(self, student_num):
        self.gene = np.array([i for i in range(0, NOB)])
        np.random.shuffle(self.gene)
        self.student_num = student_num  # 0번부터
        with open('students.txt', 'rb') as f:
            self.students = pickle.load(f)

    def remake_student(self):  # 유전자 바탕의 새로운 student 만들기
        new_student_subject = []
        target_student = self.students[self.student_num].return_sub()
        for i in self.gene:
            new_student_subject.append(target_student[i])
        new_student = student_class.Student(new_student_subject)
        self.students[self.student_num] = new_student

    def __repr__(self):
        return '<student: ' + str(self.student_num) + ', gene: ' + str(self.gene) + '>'


    def calculate_fitness(self):
        # 비용 문제는 나중에 수정
        apply_result = apply_subject.apply(self.students, NOSD, NOB, 50, False)
        self.fitness = NOB - len(apply_result[self.student_num].keys())


    def mutate(self):
        a,b = random.sample([i for i in range(0, len(self.gene))], 2)
        self.gene[a], self.gene[b] = self.gene[b], self.gene[a]



if __name__ == "__main__":

    c = Gene(1)
    c.remake_student()
    c.calculate_fitness()


<h1>genetic_alogrithm

In [3]:
class Genetic_algorithm:
    def __init__(self, student_num, population=30, mutation_chance=0.015):
        self.population = population
        self.mutation_chance = mutation_chance
        self.gene_list = []
        self.student_num = student_num

    def reset(self):
        for i in range(0, self.population):
            self.gene_list.append(Gene(self.student_num))

    def evolution(self):
        print('initial :' + str(self.mean_fitness()))
        fit_list = []
        
        for _ in range(1, 101):
            self.select()

            self.cross()

            self.mutate()
            
            mean = self.mean_fitness()
            print('prograss ' + str(_) + '/100\tfitness : ' + str(mean) )
            
            fit_list.append(mean)
        
        return fit_list
        
            
    

    def select(self):
        self.roulette_wheel()

    def roulette_wheel(self):
        # 전처리
        self.parents = []
        gene_list = self.gene_list[:]
        self.calculate_all_fitness()
        while True:
            gene = self.roulette_wheel_onetime(gene_list)
            self.parents.append(gene.gene)  # copy를 피하기 위해 subject 형태로 저장
            #if len(self.parents) >= len(self.gene_list) / 2:
            #    return
            if len(self.parents) >= 2: return
            gene_list.remove(gene)
    
    def calculate_all_fitness(self):
        for gene in self.gene_list:
            gene.remake_student()
            gene.calculate_fitness()
        

    def roulette_wheel_onetime(self, gene_list):
        fitness_list = []
        for gene in gene_list:
            fitness_list.append(gene.fitness)

        # 비용 => fitness 변환
        while True:
            k = 3  # 선택압
            max_var, min_var = max(fitness_list), min(fitness_list)
            a = lambda x: (max_var - x) + (max_var - min_var) / (k - 1)
            fitness_list = list(map(a, fitness_list))

            # 룰렛휠 선택
            sum_fitness = sum(fitness_list)
            if sum_fitness == 0:
                print(gene_list)
                print('sum_fitness is 0')
                return gene_list[0]
            before_sum = 0
            target = random.random()
            for i in range(0, len(fitness_list)):
                fitness = fitness_list[i]
                probality = before_sum + fitness / sum_fitness
                if probality < target:
                    return gene_list[i]

    def cross(self):
        self.cycle_cross()

    def cycle_cross(self):
        for i in range(0, self.population):
            sub1, sub2 = random.sample(self.parents, 2)
            sub_list = [sub1, sub2]
            main_sub, sub_sub = sub_list[0], sub_list[1]
            gene = [-1] * 10
            while -1 in gene:
                inx_list = [i for i, value in enumerate(gene) if value == -1]
                inx = random.choice(inx_list)
                used_num = []
                while True:
                    target_num = main_sub[inx]
                    if target_num in used_num: break
                    gene[inx] = target_num
                    used_num.append(target_num)
                    inx = list(sub_sub).index(target_num)
                main_sub, sub_sub = sub_sub, main_sub
            self.gene_list[i].gene = gene

    def mutate(self):
        #전형적 변이
        mutation_chance = self.mutation_chance
        for i in range(0, len(self.gene_list)):
            if random.random() < mutation_chance:
                self.gene_list[i].mutate()
    
    def mean_fitness(self):
        self.calculate_all_fitness()
        fitness_list = []
        for gene in self.gene_list:
            fitness_list.append(gene.fitness)
        
        return round(sum(fitness_list) / len(fitness_list), 3)
        
        

<h1> test

In [4]:
temp = Genetic_algorithm(20)
temp.reset()


In [5]:
fit_list = temp.evolution()

initial :0.267
prograss 1/100	fitness : 0.533
prograss 2/100	fitness : 0.267
prograss 3/100	fitness : 0.433
prograss 4/100	fitness : 0.333
prograss 5/100	fitness : 0.433
prograss 6/100	fitness : 0.433
prograss 7/100	fitness : 0.5
prograss 8/100	fitness : 0.367
prograss 9/100	fitness : 0.3
prograss 10/100	fitness : 0.433
prograss 11/100	fitness : 0.233
prograss 12/100	fitness : 0.367
prograss 13/100	fitness : 0.167
prograss 14/100	fitness : 0.4
prograss 15/100	fitness : 0.4
prograss 16/100	fitness : 0.333
prograss 17/100	fitness : 0.367
prograss 18/100	fitness : 0.433
prograss 19/100	fitness : 0.3
prograss 20/100	fitness : 0.267
prograss 21/100	fitness : 0.333
prograss 22/100	fitness : 0.367
prograss 23/100	fitness : 0.333
prograss 24/100	fitness : 0.233
prograss 25/100	fitness : 0.4
prograss 26/100	fitness : 0.367
prograss 27/100	fitness : 0.6
prograss 28/100	fitness : 0.3
prograss 29/100	fitness : 0.333
prograss 30/100	fitness : 0.5
prograss 31/100	fitness : 0.2
prograss 32/100	fitnes

KeyboardInterrupt: 

In [6]:
import matplotlib.pyplot as plt
import numpy as np

plt.plot(fit_list)

NameError: name 'fit_list' is not defined

In [7]:
a = {1:2, 3:4}
del(a[1])
print(a)

{3: 4}
